In [ ]:
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
# import fem, nglview
import sys, os, time, Bio.PDB
data_dir = '..'
sys.path.append(data_dir)
from parse_pfam import parse_pfam
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

pdb_list = Bio.PDB.PDBList(verbose=False)
pdb_parser = Bio.PDB.PDBParser()
pfam, pdb_refs = parse_pfam(data_dir)

pfam['size'] = pfam['res'] * pfam['seq']
pfam.sort_values(by='size', ascending=False, inplace=True)
pdb_refs['res'] = pdb_refs['pdb_end'] - pdb_refs['pdb_start'] + 1
pdb_refs.sort_values(by='res', ascending=False, inplace=True)
try:
    os.makedirs(os.path.join(data_dir, 'dca_comparison', 'dca'))
    os.makedirs(os.path.join(data_dir, 'dca_comparison', 'fem'))
except:
    pass

In [ ]:
cache = True

# way too big
# PF00528

dca_msas = ['PF00015', 'PF00034', 'PF00037', 'PF00079', 'PF00126', 'PF00142',
    'PF00151', 'PF00158', 'PF00165', 'PF00239', 'PF00296', 'PF00353',
    'PF00356', 'PF00376', 'PF00384', 'PF00392', 'PF00437', 'PF00440',
    'PF00465', 'PF00480', 'PF00483', 'PF00486', 'PF00497', 'PF00512',
    'PF00529', 'PF00532', 'PF00534', 'PF00535', 'PF00563', 'PF00589',
    'PF00590', 'PF00691', 'PF00805', 'PF00881', 'PF00903', 'PF00905',
    'PF00990', 'PF00994', 'PF01011', 'PF01022', 'PF01032', 'PF01041',
    'PF01047', 'PF01128', 'PF01368', 'PF01380', 'PF01381', 'PF01420',
    'PF01464', 'PF01497', 'PF01547', 'PF01551', 'PF01555', 'PF01568',
    'PF01584', 'PF01609', 'PF01613', 'PF01614', 'PF01627', 'PF01638',
    'PF01656', 'PF01695', 'PF01751', 'PF01872', 'PF01895', 'PF01966',
    'PF01978', 'PF02082', 'PF02195', 'PF02254', 'PF02272', 'PF02310',
    'PF02321', 'PF02384', 'PF02491', 'PF02515', 'PF02525', 'PF02627',
    'PF02811', 'PF02899', 'PF02954', 'PF03358', 'PF03401', 'PF03459',
    'PF03551', 'PF03602', 'PF03721', 'PF03734', 'PF03793', 'PF03989',
    'PF04545', 'PF05016', 'PF07238', 'PF07244', 'PF07676', 'PF07702',
    'PF07715', 'PF07729', 'PF08245', 'PF08279', 'PF08281', 'PF08402',
    'PF08443', 'PF08447', 'PF09084', 'PF09278', 'PF09339', 'PF10531',
    'PF02518', 'PF03466', 'PF04542','PF04542']
# 
# + ['PF00072', 'PF00496', 'PF00593']
# + ['PF00528']

score_counter = 0
score = 0.0

size_ranks = []
for ac in dca_msas:
    fam = pfam.loc[ac]
    rank = pfam['size'].rank(ascending=False)
    size_ranks.append(rank[fam.name].astype(int))

order = np.argsort(size_ranks)[::-1]
dca_msas = np.array(dca_msas)[order]

info, comparison = [], []

bugs = open(os.path.join(data_dir, 'dca_comparison', 'bugs.txt'), 'w')

for ac_count, ac in enumerate(dca_msas):

    try:
        os.makedirs(os.path.join(data_dir, 'dca_comparison', 'fem', ac))
    except:
        pass

    print('Pfam ac:', ac, ac_count, dca_msas.shape[0])

    fam = pfam.loc[ac]

    fam_dir = os.path.join(data_dir, 'Pfam-A.full', fam.name)
    msa = np.load(os.path.join(fam_dir, 'msa.npy'))
    msa = np.array([[aa.decode('utf-8') for aa in r] for r in msa])

    size_rank = pfam['size'].rank(ascending=False)[fam.name].astype(int)
    print('size rank: %i of %i' % (size_rank, pfam.shape[0]))

    aa = np.array([np.unique(s) for s in msa])
    one_aa = np.array([len(a) == 1 for a in aa])
    two_aa = np.array([len(a) == 2 for a in aa])
    missing_aa_res = np.array(['-' in a for a in aa])
    conserved_residues = one_aa | (two_aa & missing_aa_res)

    m = np.array([len(a) for a in aa])
    m = m[~conserved_residues]
    n_residues = m.shape[0]

    # compute direct info
    w_file = os.path.join(data_dir, 'dca_comparison', 'fem', ac, 'w.npy')
    disc_file = os.path.join(data_dir, 'dca_comparison', 'fem', ac, 'disc.npy')
    time_file = os.path.join(data_dir, 'dca_comparison', 'fem', ac, 'time.npy')
    if cache and os.path.exists(w_file) and os.path.exists(disc_file):
        w = np.load(w_file)
        disc = np.load(disc_file, encoding='latin1')
        start, end = np.load(time_file)
    else:
        print('no data present, skipping')
        continue
#         start = time.time()
#         model = fem.discrete.model()
#         # svd = 'exact' if msa.shape[0] < 500 and msa.shape[1] < 100000 else 'approx'
#         # print 'msa.shape, svd', msa.shape, svd
#         svd = 'approx'
#         model.fit(msa[~conserved_residues], iters=10, overfit=False, svd=svd)
#         w = np.hstack(model.w.values())
#         disc = model.disc
#         end = time.time()
#         np.save(w_file, w)
#         np.save(disc_file, disc)
#         np.save(time_file, np.array([start, end]))
    disc_mean = np.mean([d[-1] for d in disc])
    iters = np.mean([len(d) for d in disc])
    print('discrepancy, iters:', disc_mean, iters)
    print('fit time: %.02f sec' % (end - start, ))

    direct_info_file = os.path.join(data_dir, 'dca_comparison', 'fem', ac,
                                    'di.npy')
    if cache and os.path.exists(direct_info_file):
        direct_info = np.load(direct_info_file)
    else:
        mm = np.insert(m.cumsum(), 0, 0)
        w_idx = np.vstack((mm[:-1], mm[1:])).T
        direct_info = np.zeros((n_residues, n_residues))
        for i, ii in enumerate(w_idx):
            for j, jj in enumerate(w_idx):
                p = np.exp(w[ii[0]:ii[1], jj[0]:jj[1]])
                pi, pj = p.sum(axis=1), p.sum(axis=0)
                p /= p.sum()
                pi /= pi.sum()
                pj /= pj.sum()
                direct_info[i, j] = (p * np.log(p / np.outer(pi, pj))).sum()
        np.save(direct_info_file, direct_info)

    info.append([
        ac, msa.shape[0], msa.shape[1], size_rank, disc_mean, iters,
        end - start
    ])

    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    for d in disc:
        ax[0].plot(d, 'k-', lw=0.1)
    ax[0].set_xlabel('iteration')
    ax[0].set_ylabel('discrepancy')
    scale = 1e-1 * np.abs(w).max()
    ax[1].matshow(w, cmap='seismic', vmin=-scale, vmax=scale)
    ax[1].set_title('Pfam: %s' % (fam.name, ))
    plt.savefig(
        os.path.join(data_dir, 'dca_comparison', 'fem', ac, 'model.png'))
    plt.close()

    refs = pdb_refs[pdb_refs.index.str.contains(fam.name)]

    def contact_map(ref, dist_thresh=8):
        seq = msa[:, ref.seq]
        pdb_file = pdb_list.retrieve_pdb_file(
            ref.pdb_id, pdir=fam_dir, file_format='pdb')
        if os.path.exists(pdb_file):
            chain = pdb_parser.get_structure(ref.pdb_id,
                                             pdb_file)[0][ref.chain]
            coords = np.array([a.get_coord() for a in chain.get_atoms()])
            coords = coords[ref.pdb_start - 1:ref.pdb_end]
            missing_aa_seq = seq == '-'
            # if coords.shape[0] > conserved_residues.shape[0]:
            if coords.shape[0] == conserved_residues[~missing_aa_seq].shape[0]:
                coords = coords[~conserved_residues[~missing_aa_seq]]
                return distance_matrix(coords, coords) < dist_thresh

    def roc(x, c, k=5):
        mask = np.triu(np.ones(x.shape[0], dtype=bool), k=k)
        order = x[mask].argsort()[::-1]
        c_flat = c[mask][order]
        tp = np.cumsum(c_flat, dtype=float)
        fp = np.cumsum(~c_flat, dtype=float)
        tp /= tp[-1]
        fp /= fp[-1]
        return fp, tp

    dca = pd.read_csv(
        os.path.join(data_dir, 'dca_comparison', 'dca',
                     '%s_full_dca.txt' % (ac, )),
        sep=' ',
        names=['res1', 'res2', 'MI', 'DI'],
        header=None)
    n = dca.res2.max()
    dca.res1 -= 1
    dca.res2 -= 1

    dca_mi = coo_matrix((dca.MI, (dca.res1, dca.res2)), shape=(n, n)).toarray()
    dca_di = coo_matrix((dca.DI, (dca.res1, dca.res2)), shape=(n, n)).toarray()

    if direct_info.shape[0] != dca_mi.shape[0] - conserved_residues.sum():
        print('SHAPE MISMATCH 1')
        bugs.write('SHAPE MISMATCH 1 %s %i %i %i' %
                   (ac, direct_info.shape[0], dca_mi.shape[0],
                    conserved_residues.sum()))
        continue

    for i in range(refs.shape[0]):
        ref = refs.iloc[i]

        contacts = contact_map(ref)
        if contacts is None:
            print('CONTACT MAP FAIL', ref)
            bugs.write('CONTACT MAP FAIL\n % s' % (ref, ))
            continue

        compare = [
            ac, ref.seq, ref.pdb_id, ref.chain, ref.pdb_start, ref.pdb_end, 0,
            0, 0
        ]

        seq = msa[:, ref.seq]
        missing_aa_seq = seq == '-'
        idx = np.arange((~conserved_residues).sum())
        idx = idx[~missing_aa_seq[~conserved_residues]]

        top = 20
        k = 5
        x = direct_info[np.ix_(idx, idx)]
        mask = np.triu(np.ones(x.shape[0], dtype=bool), k=k)
        thresh = np.sort(np.abs(x)[mask])[-top]
        x[~mask] = 0
        predicted_contacts = np.where(x >= thresh)
        print(predicted_contacts)

        fig, ax = plt.subplots(1, 2, figsize=(8, 4))
        scale = 1e-1 * np.abs(direct_info).max()
        ax[0].matshow(direct_info, cmap='seismic', vmin=0, vmax=scale)
        ax[0].set_title('direct info')
        ax[1].matshow(contacts, cmap='Greys', aspect='auto')
        ax[1].scatter(*predicted_contacts[::-1], c='r')
        ax[1].set_title('%s%s, %i-%i (%.1f%%)' %
                        (ref.pdb_id, ref.chain, ref.pdb_start, ref.pdb_end,
                         100 * contacts[predicted_contacts].mean()))
        for a in ax:
            a.axis('off')
        plt.savefig(
            os.path.join(data_dir, 'dca_comparison', 'fem', ac, '%s%s_%i-%i' %
                         (ref.pdb_id, ref.chain, ref.pdb_start, ref.pdb_end)))
        plt.close()

        for i, x in enumerate([dca_mi, dca_di, direct_info]):
            fp, tp = roc(x[np.ix_(idx, idx)], contacts, k=k)
            auc = tp.sum() / tp.shape[0]
            compare[-3 + i] = auc

        score *= score_counter / float(score_counter + 1)
        score += np.argsort(compare[-3:])[-1] / float(score_counter + 1)
        score_counter += 1

        print(compare, score)
        comparison.append(compare)

np.save(os.path.join(data_dir, 'dca_comparison', 'fem', 'info.npy'), info)
np.save(os.path.join(data_dir, 'dca_comparison', 'comparison.npy'), comparison)
bugs.close()

In [ ]:
len(compairson)

In [ ]:
auc = np.array([row[-3:] for row in comparison])
# MI
plt.hist(auc[:,0], label='MI', color='green', alpha=0.65)
# DCA DI
plt.hist(auc[:,1], label='DCA DI', color='red', alpha=0.65)
# FEM DI
plt.hist(auc[:,2], label='FEM DI', color='blue', alpha=0.65)

plt.title('ROC AUC Comparison between MI, DCA, and FEM')
plt.xlabel('Average ROC AUC')
plt.ylabel('Number of PDBs')


plt.legend()
plt.show()
auc.mean(0)